In [32]:
import numpy as np
import pandas as pd
import datetime as dt
cols = ['ORIG','DEST','SALE_STR_DATE','TRAVEL_STR_DATE','SEASON_STR_DATE','MIN_STAY_VAL',
        'MAX_STAY_VAL','AP_VALUE','RBD']
for i in range(1,8):
    cols.append('DAY_'+str(i))
path = 'test_10000_scaled.csv'
df = pd.read_csv(path, usecols=cols, keep_default_na=False)

In [33]:
y_path = 'test_10000_Y.csv'
y_df = pd.read_csv(y_path, usecols=['FAREAMT_BC'], keep_default_na=False)


In [34]:
y_df.head()

,FAREAMT_BC
0,1036
1,1744
2,1613
3,1374
4,888


In [49]:
df.head()

,ORIG,DEST,SALE_STR_DATE,TRAVEL_STR_DATE,SEASON_STR_DATE,MIN_STAY_VAL,MAX_STAY_VAL,AP_VALUE,RBD,DAY_1,DAY_2,DAY_3,DAY_4,DAY_5,DAY_6,DAY_7
0,15,36,59,59,228,7.000000,262800,4449.313287,3,1,1,1,1,0,0,1
1,15,78,144,144,195,6.533127,525600,4449.313287,7,0,0,0,1,1,1,1
2,38,2,59,59,242,6.533127,525600,4449.313287,7,1,1,1,1,0,0,1
3,37,62,59,59,186,7.000000,262800,4449.313287,0,1,1,1,1,0,0,1
4,42,52,59,59,187,7.000000,262800,4449.313287,0,1,1,1,1,0,0,1


In [37]:
#default values to 0
for feature in cols:
    for i in range(df[feature].size):
        is_str = isinstance((df.loc[i,feature]), str)
        if is_str == True:
            print("NOT A NUMBER!",df.loc[i,feature])
            df.loc[i,feature] = int(df.loc[i,feature])
for feature in ['FAREAMT_BC']:
    for i in range(y_df[feature].size):
        is_str = isinstance((y_df.loc[i,feature]), str)
        if is_str == True:
            print("NOT A NUMBER!",y_df.loc[i,feature])
            y_df.loc[i,feature] = float(y_df.loc[i,feature])

In [38]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [50]:
X = df.values
Y = y_df.values

print(X[1])

[  1.50000000e+01   7.80000000e+01   1.44000000e+02   1.44000000e+02
   1.95000000e+02   6.53312659e+00   5.25600000e+05   4.44931329e+03
   7.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00]


In [51]:
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(16, input_dim=16, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [45]:
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=1, batch_size=5, verbose=0)

In [42]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

ValueError: Error when checking input: expected dense_23_input to have shape (None, 9) but got array with shape (9000, 16)

In [ ]:
#check for NaN and empty